In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from datetime import datetime
import os


def setup_driver():
    options = Options()
    # options.add_argument("--headless")  # Activa si no necesitas ver el navegador
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton_aceptar = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton_aceptar.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(2)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")


def cerrar_popup(driver):
    try:
        popup_close = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.modal__close"))
        )
        popup_close.click()
        print("❌ Modal cerrado.")
        time.sleep(1)
    except:
        pass


def scrape_alquiler_madrid():
    base_url = "https://www.pisos.com/venta/estudios-madrid/"
    resultados = []
    driver = setup_driver()
    wait = WebDriverWait(driver, 10)

    for page_num in range(1, 3):  # De la 1 a la 100
        url = base_url if page_num == 1 else f"{base_url}{page_num}/"
        print(f"\n🔍 Procesando página {page_num}: {url}")
        driver.get(url)

        if page_num == 1:
            aceptar_cookies(driver)

        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.ad-preview")))
            anuncios = driver.find_elements(By.CSS_SELECTOR, "div.ad-preview")
            print(f"📦 Anuncios detectados: {len(anuncios)}")
        except:
            print("⚠️ No se detectaron anuncios.")
            continue

        for anuncio in anuncios:
            try:
                titulo = anuncio.find_element(By.CSS_SELECTOR, "a.ad-preview__title").text.strip()
                ubicacion = anuncio.find_element(By.CSS_SELECTOR, "p.ad-preview__subtitle").text.strip()
                precio = anuncio.find_element(By.CSS_SELECTOR, "span.ad-preview__price").text.strip()

                caracteristicas = anuncio.find_elements(By.CSS_SELECTOR, "p.ad-preview__char.p-sm")
                habitaciones = caracteristicas[0].text.strip() if len(caracteristicas) > 0 else ""
                baños = caracteristicas[1].text.strip() if len(caracteristicas) > 1 else ""
                metros = caracteristicas[2].text.strip() if len(caracteristicas) > 2 else ""

                link = anuncio.find_element(By.CSS_SELECTOR, "a.ad-preview__title").get_attribute("href").strip()

                resultados.append({
                    "timestamp": datetime.now().isoformat(),
                    "titulo": titulo,
                    "precio": precio,
                    "ubicacion": ubicacion,
                    "habitaciones": habitaciones,
                    "baños": baños,
                    "metros": metros,
                    "link": link
                })

                if len(resultados) % 100 == 0:
                    save_to_json(resultados, "data/alquiler_madrid_parcial.json")

            except Exception as e:
                print(f"⚠️ Error extrayendo un anuncio: {e}")
                continue

        time.sleep(3)
        cerrar_popup(driver)

    driver.quit()
    return resultados



def save_to_json(data, filename="data/alquiler_madrid_completo_estudios.json"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Archivo guardado: {filename} ({len(data)} registros)")


if __name__ == "__main__":
    try:
        datos = scrape_alquiler_madrid()
        print(f"\n📊 Total de inmuebles extraídos: {len(datos)}")
        save_to_json(datos)
    except Exception as e:
        print(f"❌ Error durante el scraping: {e}")


🔍 Procesando página 1: https://www.pisos.com/venta/estudios-madrid/
🍪 Cookies aceptadas.
📦 Anuncios detectados: 31

🔍 Procesando página 2: https://www.pisos.com/venta/estudios-madrid/2/
📦 Anuncios detectados: 14

📊 Total de inmuebles extraídos: 45

💾 Archivo guardado: data/alquiler_madrid_completo_estudios.json (45 registros)
